In [1]:
from functools import reduce
import vyper
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from vyper.user import Model
from vyper.utils.tools import StatisticalTools as st
from vyper.user import Model
from vyper.user.explorer import DataProfiler
from vyper.utils.tools import StatisticalTools as st
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from pandas.api.types import is_string_dtype
from openpyxl import Workbook
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
import math
import warnings
import string

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference, LineChart
from openpyxl.drawing.text import CharacterProperties
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font

In [6]:
def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
                     variable_order, other_segment = False, file = 'Profile', exclude = '',
                     PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:
    

    profile_data = pd.DataFrame(profile_data)
    
    if (segments is None):
        segments = pd.unique(profile_data[segment_var])
        other_segment = False
        segment_names = str(segments)
    if (not (isinstance(other_segment, bool))):
        raise ValueError('other must be True or False.')
    if (segment_names is None):
        segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    if (len(segments)+other_segment!=len(segment_names)):
        if(len(segments)==len(segment_names)):
            segment_names = [segment_names, 'other']
        else:
            warnings.warn('Incorrect length of names, replacing by segments')
            segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    ## output -> [1 2 0]

    varclass = pd.DataFrame()
    for col in profile_data.columns:
        varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)
        
    # Continuous Variables that were classified as categorical
    varclass[varclass.columns in continuous] = 'continuous'
    if (not (include is None)):
        exclude = varclass.columns[varclass.columns not in include]
    # Variables to exclude
    varclass[varclass.columns in exclude] = 'exclude'
    if (excludeother): 
        varclass[varclass.columns =='Categorical+other'] = 'exclude'
    continuous_var_cuts = dict()

    
    for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
        if not continuous_var_bounds:
            continuous_var_bounds = dict()
        if variable in continuous_var_bounds:
            cut_bins = continuous_var_bounds[variable]
            continuous_var_cuts[variable] = pd.cut(profile_data[variable], bins=cut_bins)
                
        else:
            continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,squash_extremes=False)

    col_order = []
    index_order = []
    ### TODO: array([2, 0, 1]) Always encode 0 as US
    ### Change the US population as BASELINE
    for seg in segments:
        profile_1 = DataProfiler(profile_data[profile_data[segment_var] == seg]).create_profile(number_of_bins = 10,
                                                                            cts_cuts = continuous_var_cuts)
        if (seg == 'Baseline'):
            profile = profile_1
            profile.columns = ["Variable", "Category",  "US_Count","US_Percent"]
            for cols in profile.columns:
                col_order.append(cols)


        else:
            profile_1.columns = ["Variable", "Category",  "Count_"+str(seg), "Percent_"+str(seg)]
            col_order.append("Count_"+str(seg))
            col_order.append("Percent_"+str(seg))
            profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')
            profile['US Population vs ' + "Segment_" + str(seg)] = (profile["US_Percent"]/profile["Percent_"+str(seg)]) * 100
            index_order.append('US Population vs ' + "Segment_" + str(seg))
            
#     for i in range(len(segments)):
#         profile_1 = DataProfiler(profile_data[profile_data[segment_var] == segments[i]]).create_profile(number_of_bins = 10,
#                                                                             cts_cuts = continuous_var_cuts)
#         if (i == 0):
#             profile = profile_1
#             profile.columns = ["Variable", "Category",  "US_Count","US_Percent"]
#             for cols in profile.columns:
#                 col_order.append(cols)


#         else:
#             profile_1.columns = ["Variable", "Category",  "Count_"+str(i), "Percent_"+str(i)]
#             col_order.append("Count_"+str(i))
#             col_order.append("Percent_"+str(i))
#             profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')
#             profile['US Population vs ' + "Segment_" + str(i)] = (profile["US_Percent"]/profile["Percent_"+str(i)]) * 100
#             index_order.append('US Population vs ' + "Segment_" + str(i))        

    profile = profile[col_order+index_order]

    for i in profile.columns:

        if ('Count' in i):
            profile[i][profile[i].isna()] = 0
        if ('Percent' in i):
            profile[i][profile[i].isna()] = 0
            
            
    if file:
        import os
        # Make Profile ####
        filesave = file + '.xlsx'
        
        if os.path.exists(filesave):
            os.remove(filesave)
        CreateXLProfile_Snap(profile, savepath = filesave)
        
        if PPT:
            wb = Workbook()

            profile['Category'] = profile['Category'].astype(str)

            all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
            # make new sheet the active sheet we are working on
            all_var_profiling_ws = wb.active

            # remove gridlines from the sheet
            all_var_profiling_ws.sheet_view.showGridLines = False

            # set border line thickness
            thick = Side(border_style="thick", color="000000")
            # medium = Side(border_style="medium", color="000000")
            thin = Side(border_style="thin", color="000000")

            # get max rows and cols of profiling df
            max_rows = profile.shape[0]
            max_cols = profile.shape[1]
            
            input_cols = list(string.ascii_uppercase)

            j = 4
            for z in range(0, len(profile)):
                if (z != 0):
                    if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
                        # create sheet
                        all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
                        # make new sheet the active sheet we are working on
                        all_var_profiling_ws = wb.active

                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                        j = 4
                    else:
                        incol = 2
                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                            incol = incol + 1
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                            if (y % 2 == 1 and y > 2):
                                current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                                incol = incol + 1
                                current_cell.font = Font(bold=True)
                                current_cell.alignment = Alignment(horizontal='center', vertical='center')
                                current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                                current_cell.value = 'Index'
                                if y == max_cols - 1:
                                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                                else:
                                    current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
                incol = 2
                for y in range(len(profile.columns)):
                    current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                    incol = incol + 1
                    current_cell.value = profile.iloc[z, y]
                    if y == 0:
                        current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                    else:
                        current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
                    if (y % 2 == 1 and y > 2):
                        current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                        incol = incol + 1
                        current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
                        if y == max_cols - 1:
                            current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                        else:
                            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

                j = j + 1
            
            filesave = file + 'PPT.xlsx'
        
            if os.path.exists(filesave):
                os.remove(filesave)
            
            wb.save(filesave)
            
    return profile

def CreateXLProfile_Snap(profile, savepath):
    wb = Workbook()

    profile['Category'] = profile['Category'].astype(str)

    # create sheet
    all_var_profiling_ws = wb.create_sheet('profile', 0)

    # make new sheet the active sheet we are working on
    all_var_profiling_ws = wb.active

    # remove gridlines from the sheet
    all_var_profiling_ws.sheet_view.showGridLines = False

    # set border line thickness
    thick = Side(border_style="thick", color="000000")
    # medium = Side(border_style="medium", color="000000")
    thin = Side(border_style="thin", color="000000")

    # get max rows and cols of profiling df
    max_rows = profile.shape[0]
    max_cols = profile.shape[1]

    input_rows = range(4, max_rows + 4)
    input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

    y = 0
    for i in input_cols:
        x = "3"
        x = i + x

        all_var_profiling_ws[x].font = Font(bold=True)
        current_cell = all_var_profiling_ws[x]
        current_cell.alignment = Alignment(horizontal='center', vertical='center')
        current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

        if y == 0:
            current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
        elif y == max_cols - 1:
            current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
        else:
            current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

        all_var_profiling_ws[x] = profile.columns[y]
        y = y + 1
    
    print("Worked till here")
    counter = []
    counter_val = 1

    for index,i in enumerate(input_cols[5::]):
        x = "2"
        x = i + x
        if index%2==1:
            all_var_profiling_ws.merge_cells(f"{counter[-1]}:{x}")
            all_var_profiling_ws[counter[-1]].font = Font(bold=True)
            current_cell = all_var_profiling_ws[counter[-1]]
            current_cell.alignment = Alignment(horizontal='center', vertical='center')
            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
            continue


        if i==input_cols[-1]:
            break
        if i=='G':
            all_var_profiling_ws[x] = "US POPULATION"
            all_var_profiling_ws.merge_cells(start_row=2, start_column=7, end_row=2, end_column=8)
            counter.append(x)
            continue
        all_var_profiling_ws[x] = 'Segment ' + str(counter_val)
        counter_val+=1
        counter.append(x)
        # all_var_profiling_ws.merge_cells(start_row=2, start_column=5, end_row=2, end_column=5)
        # all_var_profiling_ws.merge_cells(f'{x}:D2')

    
    def style_range(ws, cell_range, border=Border(), fill=None, font=None,alignment=None):
        
        """
        Apply styles to a range of cells as if they were a single cell.

        :param ws:  Excel worksheet instance
        :param range: An excel range to style (e.g. A1:F20)
        :param border: An openpyxl Border
        :param fill: An openpyxl PatternFill or GradientFill
        :param font: An openpyxl Font object
        """

        top = Border(top=border.top)
        left = Border(left=border.left)
        right = Border(right=border.right)
        bottom = Border(bottom=border.bottom)

        first_cell = ws[cell_range.split(":")[0]]
        if alignment:
            ws.merge_cells(cell_range)
            first_cell.alignment = alignment

        rows = ws[cell_range]
        if font:
            first_cell.font = font

        for cell in rows[0]:
            cell.border = cell.border + top
        for cell in rows[-1]:
            cell.border = cell.border + bottom

        for row in rows:
            l = row[0]
            r = row[-1]
            l.border = l.border + left
            r.border = r.border + right
            if fill:
                for c in row:
                    c.fill = fill


    border = Border(top=thick, left=thick, right=thick, bottom=thick)

    for range1 in all_var_profiling_ws.merged_cells.ranges:
        style_range(all_var_profiling_ws, str(range1), border=border)

    

    y = 0
    for i in input_cols:
        z = 0
        for j in input_rows:
            all_var_profiling_ws.row_dimensions[j].height = 25
            x = str(j)
            x = i + x
            # print(x)
            current_cell = all_var_profiling_ws[x]
            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

            if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

            all_var_profiling_ws[x] = profile.iloc[z, y]

            if y == 0:
                current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
            elif y == max_cols - 1:
                current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


            z = z + 1
        y = y + 1
    if (savepath != ''):
        wb.save(savepath)

In [7]:
# Read in the demonstration Epsilon Response Model Data - Internal Append
# eps_data = pd.read_csv(r"Data/EpsilonResponseModelData_InternalAppend.csv")
eps_data = pd.read_csv('Data/selected_attributes.csv')

# Read in the Epsilon Data Dictionary we created
epsilon_dict = pd.read_csv('Data/EpsilonDataDictionary.csv')

# Pull list of Selected Attributes from Snowflake Headers
variable_list = epsilon_dict['Snowflake Headers'].tolist()
# variable_list.append('seg')

# Selected Attributes only
new_df = eps_data[variable_list]
# %%
# Split the data in half just for testing purposes of dummy groups
cutoff = int(len(new_df)/2)
baseline_df = new_df.iloc[:cutoff,:]
baseline_df = new_df.iloc[:cutoff,:]
customer1_df = new_df.iloc[cutoff:,:]

# Create Dummy Groups
baseline_df['Group'] = 'Baseline'
customer1_df['Group'] = 'Customer_1'
             
# Merge the two dataframes together again
final_df = baseline_df.append(customer1_df)
final_df

,RECORD_QUALITY_CODE,NUM_SOURC_VERIFY_HH,FIPS_STATE_CODE,ZIP,ZIP4,DELIVERY_POINT_CODE,CARRIER_ROUTE,CONTRACTED_ADDRESS,POST_OFFICE_NAME,STATE,...,MT_TRADITIONAL_CABLE_PAY_TV_SERVICE_CUSTOMERS,INVESTMENT_RESOURCES_TIERS,LIQUID_RESOURCES_TIERS,MORTGAGE_LIABILITY_TIERS,SHORT_TERM_LIABILITY_TIERS,WEALTH_RESOURCES_TIERS,_2010_CENSUS_GEO_TRAC_BLOCK_GROUP,CENSUS_TRACT_SUFFIX,_2010_CENSUS_GEO_BLOCK,Group
0,1,2,36,11209,5503,653,C013,65 89TH ST,BROOKLYN,NY,...,64,10,10,360.0,7,7,50002,0,2,Baseline
1,1,21,36,11427,2143,41,C027,23404 UNION TPKE,QUEENS VLG,NY,...,2,10,12,938.0,8,6,1301001,0,3,Baseline
2,1,8,9,6880,5532,139,C018,113 REGENTS PARK,WESTPORT,CT,...,50,11,14,455.0,5,7,506003,0,4,Baseline
3,1,11,36,10028,4456,947,C021,351 E 84TH ST APT 9D,NEW YORK,NY,...,14,8,12,NaN,4,5,146014,1,0,Baseline
4,1,11,36,11357,1326,209,C006,620 160TH ST,WHITESTONE,NY,...,34,11,11,598.0,9,7,991001,0,3,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19847,1,18,34,7463,2301,121,C003,12 MAPLE CT,WALDWICK,NJ,...,8,11,12,332.0,9,7,561003,0,16,Customer_1
19848,1,5,36,11507,1123,289,C007,128 NASSAU DR,ALBERTSON,NY,...,44,11,14,655.0,7,9,3023003,0,10,Customer_1
19849,1,4,34,7760,1062,632,C005,163 RUMSON RD,RUMSON,NJ,...,76,10,12,NaN,2,7,8038004,0,20,Customer_1
19850,1,10,36,11209,6757,11,C046,9452 RIDGE BLVD APT 1,BROOKLYN,NY,...,77,9,9,NaN,7,6,54001,0,0,Customer_1


In [8]:
# Function to read in a csv with containing the variable names and the respective bin cutoffs
def continuous_bins(filepath):
    df_continuous = pd.read_csv(filepath)
    
    continuous_bounds = {}
    for index, row in df_continuous.iterrows():
        continuous_bounds[row['Snowflake Field names']] = [float(i) for i in row['Bin'].split(sep=',')]
        
    return continuous_bounds
# Set up variables for snapshot
file_name = '2-seg-20k-test'
seg_var = 'Group'
dataset = final_df

# Read in file and set bins
bin_vars = continuous_bins('Data/bin_dictionary.csv')

In [9]:
file_name = 'OneGroup'
profile = Snapshot_Profile(profile_data = dataset, segment_var=seg_var, segments = None, segment_names = None, include = None,
                 continuous_var_bounds = bin_vars, variable_order = None, other_segment = False,
                 file = file_name, exclude = [], PPT = True, continuous = [], excludeother = False)


Worked till here


In [10]:
epsilon_path = 'Epsilon_Final.xlsx'
def addextra(epsilonpath,profile):

    Field_dict = pd.read_excel(epsilonpath)
    Field_dict['NAME'].fillna(method = 'ffill',inplace = True)
    Field_dict['Snowflake'].fillna(method = 'ffill',inplace = True)
    Field_dict['Value'].fillna('',inplace = True)
    Field_dict.columns = ['NAME', 'Description', 'RATEID', 'Value', 'Value Description',
        'Current Count', 'Current %', 'Snowflake', 'Category']
    import numpy as np
    prev_dict = {}
    def toDict(Field,Value,VD,snowflake,FDdescription):
        
        if Value == '':
            return 0
            
        if snowflake not in prev_dict:
            prev_dict[snowflake] = {}
            prev_dict[snowflake]['Field'] = Field
            prev_dict[snowflake]['Field Description'] = FDdescription
        if Value not in prev_dict[snowflake]:
            prev_dict[snowflake][Value] =  {}
        prev_dict[snowflake][Value]['desp'] =  VD
        #prev_dict[snowflake][Value]['Current Count'] =  CC
        #prev_dict[snowflake][Value]['Current %'] = CP
        
        return prev_dict

    k = Field_dict.apply(lambda x: toDict(x['NAME'],x['Value'],x['Value Description'],x['Snowflake'],x['Description']),axis = 1)


    def addFieldName(x):
        if x in prev_dict.keys():
            return prev_dict[x]['Field']

    profile.insert(1,'Label','')
    profile['Label'] = profile['Variable'].apply(lambda x:addFieldName(x))

    def addFieldDesc(x):
        if x in prev_dict.keys():
            return prev_dict[x]['Field Description']
    profile.insert(2,'Definition','')
    profile['Definition'] = profile['Variable'].apply(lambda x:addFieldDesc(x))

    def addValuedescription(snowflake,value):
        if snowflake in prev_dict.keys():
            if value in prev_dict[snowflake].keys():
                return prev_dict[snowflake][value]['desp']

    profile.insert(4,'Description','')
    profile['Description'] = profile.apply(lambda x:addValuedescription(x['Variable'],x['Category']),axis = 1)


addextra('Data/Epsilon_Final.xlsx',profile)
profile


,Variable,Label,Definition,Category,Description,US_Count,US_Percent,Count_Customer_1,Percent_Customer_1,US Population vs Segment_Customer_1
0,NUM_SOURC_VERIFY_HH,Number of Sources Verifying Household,NaN,"(0.999, 4.0]",None,979.0,9.862986,1069.0,10.769696,91.580917
1,NUM_SOURC_VERIFY_HH,Number of Sources Verifying Household,NaN,"(4.0, 6.0]",None,1057.0,10.648801,1088.0,10.961112,97.150735
2,NUM_SOURC_VERIFY_HH,Number of Sources Verifying Household,NaN,"(6.0, 9.0]",None,1305.0,13.147290,1271.0,12.804755,102.675059
3,NUM_SOURC_VERIFY_HH,Number of Sources Verifying Household,NaN,"(9.0, 11.0]",None,842.0,8.482773,826.0,8.321580,101.937046
4,NUM_SOURC_VERIFY_HH,Number of Sources Verifying Household,NaN,"(11.0, 13.0]",None,826.0,8.321580,856.0,8.623816,96.495327
...,...,...,...,...,...,...,...,...,...,...
6735,VEHICLE_MILEAGE_CODE_2,Vehicle Mileage Code 2,"Code indicating the number of miles (in 10,000...",V,"210,001 - 220,000",0.0,0.000000,1.0,0.010075,NaN
6736,VEHICLE_MILEAGE_CODE_2,Vehicle Mileage Code 2,"Code indicating the number of miles (in 10,000...",W,"220,001 - 230,000",0.0,0.000000,1.0,0.010075,NaN
6737,VEHICLE_MILEAGE_CODE_2,Vehicle Mileage Code 2,"Code indicating the number of miles (in 10,000...",X,"230,001 - 240,000",0.0,0.000000,1.0,0.010075,NaN
6738,VEHICLE_SERVICE_INDICATOR_4,Vehicle Service Indicator 4,Person within the household that brought vehic...,5.0,None,0.0,0.000000,2.0,0.020149,NaN


In [11]:
profile_copy = profile
file_name = 'gg'
file_xlsx = file_name + '.xlsx'
CreateXLProfile_Snap(profile, file_xlsx)

Worked till here
